# Codeanywhere PyTorch Template

Welcome to the PyTorch on Codeanywhere! This template is your springboard for creating amazing image classification projects. We'll use the FashionMNIST dataset – a stylish alternative to the classic MNIST digits – to get you started.

In this interactive notebook, you'll learn how to:

*   **Load a Dataset:** Grab FashionMNIST and prepare it for our model.
*   **Build a Neural Network:** Create a simple but effective network to recognize clothing items.
*   **Train Your Model:** Teach your network to identify different types of clothes with a training loop.
*   **Test It Out:** See your model in action by testing it on new images. How well does it perform?

Ready to become an AI engineer? Let's dive in!

The code is based on the [Pytorch build a model tutorial](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html). You can find more help on the links below:

[Learn the Basics](intro.html) ||
[Quickstart](quickstart_tutorial.html) ||
[Tensors](tensorqs_tutorial.html) ||
[Datasets & DataLoaders](data_tutorial.html) ||
[Transforms](transforms_tutorial.html) ||
**Build Model** ||
[Autograd](autogradqs_tutorial.html) ||
[Optimization](optimization_tutorial.html) ||
[Save & Load Model](saveloadrun_tutorial.html)

# Building the Neural Network

Neural networks are made up of layers/modules that perform operations on data. The [torch.nn](https://pytorch.org/docs/stable/nn.html) namespace provides all the building blocks you need to create your own neural networks. Every module in PyTorch subclasses the [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html). A neural network itself is a module consisting of other modules (layers). This nested structure makes it easy to build and manage complex architectures.

In the following sections, we'll build a neural network to classify images in the FashionMNIST dataset.

In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import random

## Getting a Device
We want to train our model on a hardware accelerator like the GPU, if available. Let's check if [torch.cuda](https://pytorch.org/docs/stable/notes/cuda.html) or [torch.backends.mps](https://pytorch.org/docs/stable/notes/mps.html) are available; otherwise, we'll use the CPU. Codeanywhere may not have GPU support, but we're always looking to improve. Keep an eye out for future updates!


In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

## Defining Our Neural Network
We define our neural network by subclassing `nn.Module` and initializing the neural network layers in `__init__`. Every `nn.Module` subclass implements the operations on input data in the `forward` method.

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

Let's create an instance of `NeuralNetwork`, move it to our `device`, and print its structure.

In [ ]:
model = NeuralNetwork().to(device)
print(model)

To use the model, we pass it the input data. This executes the model's `forward` method, along with some [background operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866). Do not call `model.forward()` directly!

Calling the model on the input returns a 2-dimensional tensor with dim=0 corresponding to each output of 10 raw predicted values for each class, and dim=1 corresponding to the individual values of each output. We get the prediction probabilities by passing it through an instance of the `nn.Softmax` module.

In [ ]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

## Let's dive deeper...

## Inside the Model Layers

Let's break down the layers in the FashionMNIST model. To illustrate, we'll take a minibatch of 3 images of size 28x28 and see what happens as we pass it through the network.

In [ ]:
input_image = torch.rand(3,28,28)
print(input_image.size())

### nn.Flatten
We initialize the [nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html) layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values. The minibatch dimension (at dim=0) is maintained.

In [ ]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

### nn.Linear
The [linear layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) is a module that applies a linear transformation on the input using its stored weights and biases.

In [ ]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

### nn.ReLU
Non-linear activations create complex mappings between the model's inputs and outputs. They're applied after linear transformations to introduce *nonlinearity*, helping neural networks learn a wide variety of phenomena.

In this model, we use [nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html) between our linear layers, but there are other activations to introduce non-linearity in your model.

In [ ]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

### nn.Sequential
[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) is an ordered container of modules. The data is passed through all the modules in the same order as defined. You can use sequential containers to quickly assemble a network like `seq_modules`.

In [ ]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

### nn.Softmax
The last linear layer of the neural network returns `logits` - raw values in [-\infty, \infty] - which are passed to the [nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html) module. The logits are scaled to values [0, 1], representing the model's predicted probabilities for each class. The `dim` parameter indicates the dimension along which the values must sum to 1.

In [ ]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

## Model Parameters
Many layers inside a neural network are *parameterized*, meaning they have associated weights and biases that are optimized during training. Subclassing `nn.Module` automatically tracks all fields defined inside your model object and makes all parameters accessible using your model's `parameters()` or `named_parameters()` methods.

In this example, we iterate over each parameter and print its size and a preview of its values.

In [ ]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

## Training Time!
Now let's set up the training loop. First, we'll need a few more things:

In [ ]:
learning_rate = 1e-3
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Define the loop functions

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()  # Set the model to training mode
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval() # Set the model to evaluation mode
    test_loss, correct = 0, 0
    with torch.no_grad(): # Disable gradient calculation during inference
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Now download and prepare the dataset


In [ ]:
# Define the transformations
transform = transforms.Compose([
    transforms.ToTensor(), # Convert images to tensors
    transforms.Normalize((0.5,), (0.5,)) # Normalize pixel values
])

# Download the training and testing datasets
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=transform
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=transform
)

# Create data loaders
batch_size = 64 # Feel free to play with this

train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

#Let's see a sample of the training data
labels_map = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(0, len(training_data), size=(1,)).item()
    img, label = training_data[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(labels_map[label])
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")
plt.show()

## Run the Training


In [ ]:
epochs = 5 #A typical training will involve a lot more epochs, but for demonstration purposes i'll keep it short
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

## Seeing the Results (Inference)

Now that we have a trained model, let's see how it performs on some new images from the test dataset. We'll pick a random image and see what the model predicts.

In [ ]:
# Get a batch of test images
X, y = next(iter(test_dataloader))
X, y = X.to(device), y.to(device)

# Make predictions
model.eval() #Set the model to evaluation mode
with torch.no_grad():
    pred = model(X)
    predicted, actual = labels_map[pred.argmax(1)[0].item()], labels_map[y[0].item()]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

plt.imshow(X[0].cpu().squeeze(), cmap="gray")
plt.show()

## Saving the Model (Persistence)

Let's save our trained model so we can use it later without retraining.

In [ ]:
torch.save(model.state_dict(), "fashion_model.pth")
print("Saved PyTorch Model State to fashion_model.pth")

## Wrapping Up!

Congratulations! You've built and trained a fashion classification model using PyTorch and Codeanywhere. This is just the beginning. Experiment with different network architectures, training parameters, and datasets to create even more amazing AI applications!

- [torch.nn API](https://pytorch.org/docs/stable/nn.html)